##  07.8 - Learning Policy on Image + Attributes - loss 2

Fully connected network with:
- Input: 
    - **184** size vector with:
        * client information: [current latitude, current longitud, destination latitude, destination longitude]
        * vehicles information: [load, queue, current latitude, current longitud, next latitude, next longitude]
    - Image with 31 channel: 
        * Channel 0 --> client representation. its current location is marked as 1; its destination as -1
        * Channels > 0 --> vehicle current location (1)
        
- Output: One hot encoding vector with 30 entries representing the available vehicles


The attribute input is merged later with a partial output of the network

loss 1: vehicle assignment
loss 2: vehicle position

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import random
import tqdm
import time

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch import optim

In [36]:
class ImagesP3(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(2, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 4*(self.cs-1))*(im_size - 4*(self.cs-1)), 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(300, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1] 
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv4(self.conv3(x2))
        
        x4 = x3.view(-1, 2*(x.shape[-1] - 4*(self.cs-1))*(x.shape[-1] - 4*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        # add vehicles information
        x7 = torch.cat([v_x.transpose(2,1) ,x6.view(-1, 1,v_x.shape[1] )], dim=1).view(v_x.shape[0], -1)
        
        x8 = self.fc3(x7)
        x9 = self.fc4(x8)
        
        return x_aux, x9

In [11]:
def epoch_train(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size=30, simple=False, weighted=0.5):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = criterion1(output1, test_y_aux) + criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2
    

### General configurations

In [5]:
random.seed(4915)
global MAX_V, mini_batch_size, n_epochs, lr, inp_size
MAX_V = 30
mini_batch_size = 50
n_epochs=40

tables = list(range(1,90))
random.shuffle(tables)

lr = 0.0001
inp_size = 4 + MAX_V*6

train_tables, test_tables, validation_tables = \
            tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.9)], \
            tables[int(len(tables)*0.9):]

In [ ]:
model = ImagesP3(80, MAX_V, 100)



In [ ]:
print('Total num of parameters: {}'.format(sum(p.numel() for p in model.parameters())))
print('Num of trainable parameters: {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

### Functions

In [6]:
def evaluating_model(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2


def visualize_results(loss, test_loss, acc, test_acc, idx_f):
    f, ax = plt.subplots(2, 1, figsize=(20,8))
    ax[0].plot(loss)
    ax[0].plot(test_loss)
    ax[1].plot(acc)
    ax[1].plot(test_acc)
    plt.show()
    
    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc) 
    iter_max = iter_max[0][0]

    fig, ax = plt.subplots(1,5, figsize=(20,4))
    k = 0
    idx = idx_f[:iter_max+1]
    for j, i in enumerate(idx[::-1]):
        if len(i) > 0:
            ax[k].hist(i, bins=50)
            ax[k].set_xlabel('Client')
            ax[k].set_ylabel('Num errors')
            ax[k].set_title('Iteration {}\nwith acc:{:.3f}'.format(-j + iter_max, test_acc[-j + iter_max]))
            k += 1
        if k == 5:
            plt.show()
            break

### Influence of kernel size

In [16]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3, 5, 10]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}'.format(np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
    

Epoch 40. Train Loss: 3.959 Accuracy: 0.532 Test Loss: 169.891 Accuracy: 0.187
Average time per epoch 11.082s +- 1.950
Max accuracy of 0.201 achieved at epoch 22
Epoch 40. Train Loss: 4.084 Accuracy: 0.552 Test Loss: 169.746 Accuracy: 0.191
Average time per epoch 18.875s +- 59.310
Max accuracy of 0.201 achieved at epoch 24
Epoch 40. Train Loss: 4.595 Accuracy: 0.533 Test Loss: 167.956 Accuracy: 0.194
Average time per epoch 10.735s +- 0.168
Max accuracy of 0.204 achieved at epoch 25
Epoch 40. Train Loss: 4.018 Accuracy: 0.532 Test Loss: 169.715 Accuracy: 0.181
Average time per epoch 10.723s +- 0.136
Max accuracy of 0.194 achieved at epoch 28
Epoch 40. Train Loss: 3.982 Accuracy: 0.541 Test Loss: 170.716 Accuracy: 0.181
Average time per epoch 10.802s +- 0.190
Max accuracy of 0.193 achieved at epoch 31
Average accuracy 0.199 +- 0.004. Av loss 168.667
Epoch 40. Train Loss: 4.348 Accuracy: 0.509 Test Loss: 169.576 Accuracy: 0.196
Average time per epoch 12.163s +- 0.147
Max accuracy of 0.203

## Influence of Batch Norm in Fully connected layer

In [18]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}'.format(np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
    

Epoch 40. Train Loss: 4.933 Accuracy: 0.171 Test Loss: 169.134 Accuracy: 0.125
Average time per epoch 11.069s +- 0.641
Max accuracy of 0.125 achieved at epoch 39
Epoch 40. Train Loss: 4.527 Accuracy: 0.179 Test Loss: 148.015 Accuracy: 0.137
Average time per epoch 11.087s +- 0.417
Max accuracy of 0.138 achieved at epoch 37
Epoch 40. Train Loss: 4.519 Accuracy: 0.176 Test Loss: 164.081 Accuracy: 0.132
Average time per epoch 11.139s +- 0.294
Max accuracy of 0.133 achieved at epoch 38
Epoch 40. Train Loss: 5.202 Accuracy: 0.189 Test Loss: 164.299 Accuracy: 0.132
Average time per epoch 11.195s +- 0.414
Max accuracy of 0.133 achieved at epoch 38
Epoch 40. Train Loss: 5.327 Accuracy: 0.182 Test Loss: 170.301 Accuracy: 0.131
Average time per epoch 11.124s +- 0.224
Max accuracy of 0.131 achieved at epoch 39
Average accuracy 0.132 +- 0.004. Av loss 163.075


## Influence of layer size -- from 100 to 300

In [20]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
    

Epoch 40. Train Loss: 3.907 Accuracy: 0.768 Test Loss: 172.820 Accuracy: 0.183
Average time per epoch 11.987s +- 0.659
Max accuracy of 0.213 achieved at epoch 11
Epoch 40. Train Loss: 3.300 Accuracy: 0.754 Test Loss: 174.079 Accuracy: 0.180
Average time per epoch 11.970s +- 0.357
Max accuracy of 0.201 achieved at epoch 17
Epoch 40. Train Loss: 3.469 Accuracy: 0.777 Test Loss: 174.829 Accuracy: 0.170
Average time per epoch 12.012s +- 0.169
Max accuracy of 0.195 achieved at epoch 7
Epoch 40. Train Loss: 3.527 Accuracy: 0.774 Test Loss: 173.471 Accuracy: 0.185
Average time per epoch 12.136s +- 0.481
Max accuracy of 0.209 achieved at epoch 11
Epoch 40. Train Loss: 3.852 Accuracy: 0.767 Test Loss: 173.270 Accuracy: 0.182
Average time per epoch 12.275s +- 0.470
Max accuracy of 0.199 achieved at epoch 17
Average accuracy 0.204 +- 0.006. Av loss 169.756
 -------------


## Different architectures

In [27]:
print(model)

ImagesP3(
  (conv1): Sequential(
    (0): Conv2d(31, 20, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(20, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(10, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Linear(in_features=1152, out_features=264, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=264, out_features=30, bias=True)
  )
  (f_aux): Sequential(
    (0): Linear(in_features=30, out_features=2, bias=True)
  )
  (fc3): Sequential(
    (0): Linear(in_features=210, out_features=300, bias=True)
    (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=Tr

In [23]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 3.457 Accuracy: 0.776 Test Loss: 173.242 Accuracy: 0.179
Average time per epoch 16.937s +- 1.201
Max accuracy of 0.196 achieved at epoch 25
Epoch 40. Train Loss: 3.197 Accuracy: 0.774 Test Loss: 173.496 Accuracy: 0.173
Average time per epoch 17.093s +- 0.576
Max accuracy of 0.196 achieved at epoch 10
Epoch 40. Train Loss: 3.176 Accuracy: 0.775 Test Loss: 174.008 Accuracy: 0.177
Average time per epoch 16.982s +- 0.796
Max accuracy of 0.198 achieved at epoch 9
Epoch 40. Train Loss: 3.253 Accuracy: 0.788 Test Loss: 174.502 Accuracy: 0.173
Average time per epoch 17.325s +- 0.417
Max accuracy of 0.207 achieved at epoch 14
Epoch 40. Train Loss: 3.262 Accuracy: 0.770 Test Loss: 174.042 Accuracy: 0.172
Average time per epoch 17.365s +- 0.626
Max accuracy of 0.200 achieved at epoch 17
Average accuracy 0.199 +- 0.004. Av loss 169.389
 -------------


In [37]:
model = ImagesP3(31, MAX_V, im_size, kernel_size)

In [38]:
print(model)

ImagesP3(
  (conv1): Sequential(
    (0): Conv2d(31, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(5, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Linear(in_features=968, out_features=264, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=264, out_features=30, bias=True)
  )
  (f_aux): Sequential(
    (0): Linear(in_featur

In [39]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 4.649 Accuracy: 0.760 Test Loss: 173.698 Accuracy: 0.170
Average time per epoch 10.527s +- 0.880
Max accuracy of 0.202 achieved at epoch 10
Epoch 40. Train Loss: 4.082 Accuracy: 0.751 Test Loss: 174.303 Accuracy: 0.175
Average time per epoch 10.753s +- 0.222
Max accuracy of 0.198 achieved at epoch 12
Epoch 40. Train Loss: 3.410 Accuracy: 0.770 Test Loss: 173.768 Accuracy: 0.169
Average time per epoch 10.711s +- 0.361
Max accuracy of 0.203 achieved at epoch 11
Epoch 40. Train Loss: 3.789 Accuracy: 0.757 Test Loss: 171.505 Accuracy: 0.181
Average time per epoch 10.452s +- 0.495
Max accuracy of 0.199 achieved at epoch 9
Epoch 40. Train Loss: 3.455 Accuracy: 0.758 Test Loss: 173.542 Accuracy: 0.177
Average time per epoch 10.694s +- 0.483
Max accuracy of 0.197 achieved at epoch 12
Average accuracy 0.200 +- 0.002. Av loss 169.241
 -------------


## FUTURE LOCATION AS -1

In [42]:
class ImagesVal2(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))*(im_size - 3*(self.cs-1)), 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(300, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1]
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = x3.view(-1, 2*(x.shape[-1] - 3*(self.cs-1))*(x.shape[-1] - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        # add vehicles information
        x7 = torch.cat([v_x.transpose(2,1) ,x6.view(-1, 1,v_x.shape[1] )], dim=1).view(v_x.shape[0], -1)
        
        x8 = self.fc3(x7)
        x9 = self.fc4(x8)
        
        return x_aux, x9

In [43]:
def epoch_train2(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size=30, simple=False, weighted=0.5):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = criterion1(output1, test_y_aux) + criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2

def evaluating_model(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train2(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [44]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesVal2(MAX_V, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 3.061 Accuracy: 0.771 Test Loss: 172.059 Accuracy: 0.187
Average time per epoch 12.004s +- 0.349
Max accuracy of 0.213 achieved at epoch 17
Epoch 40. Train Loss: 4.349 Accuracy: 0.758 Test Loss: 173.645 Accuracy: 0.178
Average time per epoch 11.686s +- 0.225
Max accuracy of 0.201 achieved at epoch 10
Epoch 40. Train Loss: 3.505 Accuracy: 0.767 Test Loss: 174.196 Accuracy: 0.163
Average time per epoch 11.456s +- 0.136
Max accuracy of 0.194 achieved at epoch 10
Epoch 40. Train Loss: 3.483 Accuracy: 0.761 Test Loss: 174.070 Accuracy: 0.177
Average time per epoch 11.472s +- 0.146
Max accuracy of 0.194 achieved at epoch 16
Epoch 40. Train Loss: 3.452 Accuracy: 0.774 Test Loss: 174.474 Accuracy: 0.173
Average time per epoch 11.458s +- 0.119
Max accuracy of 0.198 achieved at epoch 13
Average accuracy 0.200 +- 0.007. Av loss 169.679
 -------------


In [48]:
def epoch_train3(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size=30, simple=False, weighted=0.5):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), 2*nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][2*i-1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][2*i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), 2*nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][2*i-1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][2*i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = criterion1(output1, test_y_aux) + criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2

def evaluating_model3(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train3(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [49]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesVal2(2*MAX_V+1, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model3(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 2.998 Accuracy: 0.774 Test Loss: 173.151 Accuracy: 0.165
Average time per epoch 13.949s +- 0.576
Max accuracy of 0.198 achieved at epoch 12
Epoch 40. Train Loss: 3.928 Accuracy: 0.756 Test Loss: 173.804 Accuracy: 0.173
Average time per epoch 14.244s +- 0.580
Max accuracy of 0.202 achieved at epoch 11
Epoch 19. Train Loss: 5.175 Accuracy: 0.538 Test Loss: 170.262 Accuracy: 0.194

KeyboardInterrupt: 